In [1]:
import numpy as np
import pandas as pd
from scipy.spatial.distance import cdist

import matplotlib.pyplot as plt
plt.rc("figure", figsize=(10,4))
import seaborn as sns
sns.set_context("paper", font_scale=1.3)

import poissonlearning as pl

import storage
import plotting

plotting.setup(latex=True)

%load_ext autoreload
%autoreload 2

%matplotlib qt
sns.set_context("paper", font_scale=1.3)

In [2]:
SAVE_PLOTS = True
NUM_PLOTTING_POINTS = 5000

### Convenience functions

In [3]:
def compute_errors(experiments):
    for experiment in experiments:
        solution = experiment["solution"]
        z = solution["z"].to_numpy()

        reference_experiments = list(
            filter(
                lambda x: 
                    x["seed"] == experiment["seed"] 
                    and x["n"] == experiment["n"]
                    and x["bump"] == experiment["bump"]
                    and x["eps"] == experiment["eps"],
                experiments
            )
        )
        reference = max(reference_experiments, key=lambda x: x["p"])["solution"]

        error_L1 = np.abs(z - reference["z"]).mean()
        experiment["error_L1"] = error_L1

In [4]:
experiments = storage.load_results(name="p_one_circle", folder="../results")
compute_errors(experiments)

### Visualisation for various p

In [5]:
selected_experiments = list(
    filter(
        lambda x: 
            x["seed"] == 0 
            and x["n"] == 10000
            and x["bump"] == "dirac",
        experiments
    )
)
selected_experiments.sort(key=lambda x: x["p"])

In [6]:
sample_size = NUM_PLOTTING_POINTS

fig_results = plt.figure()
for i, ex in enumerate(selected_experiments, start=1):
    ax_solution = fig_results.add_subplot(
        4,  # int(np.floor(np.sqrt(len(results[n])))),
        6,  # int(np.floor(np.sqrt(len(results[n])))),
        i,
        projection="3d",
    )

    sample = ex["solution"].sample(sample_size, random_state=1)
    xy = sample[["x", "y"]].to_numpy()

    dist = cdist(xy, xy, metric="euclidean",)
    plotting.plot_graph_function_with_triangulation(
        ax_solution, xy, sample["z"].to_numpy(), dist=dist, max_dist=0.1,
    )
    ax_solution.set_title(f"p={ex['p']}; n={ex['n']}")
fig_results.tight_layout()

### Plot of error to largest p

In [7]:
n = 30000
selected_experiments = {
    n : list(
        filter(
            lambda x: 
                x["n"] == n
                and x["bump"] == "dirac",
            experiments
        )
    )
    for n in [10000, 20000, 30000]
}

p_error = list(set([e["p"] for e in list(selected_experiments.values())[0]]))
p_error = sorted(p_error)
p_max = max(p_error)
error = {bump: {} for bump in selected_experiments.keys()}
for bump, experiments_with_bump in selected_experiments.items():
    for p in p_error:
        if np.isclose(p, p_max):
            continue

        ex = list(filter(lambda x: np.isclose(x["p"], p), experiments_with_bump))
        error[bump][p] = {}
        error[bump][p]["mean"] = np.mean([e["error_L1"] for e in ex])
        error[bump][p]["max"] = np.max([e["error_L1"] for e in ex])
        error[bump][p]["min"] = np.min([e["error_L1"] for e in ex])

In [11]:
plt.rc("figure", figsize=(10,4))
sns.set_context("paper", font_scale=1.3)
fig, ax = plt.subplots(1, 1)

colors = plotting.get_plot_colors(len(selected_experiments))
for i, (bump, e) in enumerate(error.items()):
    plotting.error_plot({bump: e}, ax, c=colors[i])

ax.legend()
ax.set_xlabel(fr"$p$")
ax.set_ylabel(r"$\lVert u^p - u^{100} \rVert_1$")
ax.set_ylim(0.0, None)

fig.tight_layout()

if SAVE_PLOTS:
    fig.savefig(f"../plots/p_one_circle_error_largest_p.pdf", bbox_inches="tight")